In [164]:
import numpy as np
import pandas as pd
from datetime import datetime
from alex.database import Database # вспомогательный класс для подключения к базе и работе с sql

In [165]:
db = Database()

In [166]:
# https://github.com/twmeggs/PyRebuildLOB/blob/master/rebuilding_LOB.pdf

In [167]:
tick = 19 # binance, ETH/USDT
lag_hours = 48

In [168]:
# 5 sec
df = db.query(f"select * from tvf_get_HOB({tick}, dateadd(hour, -{lag_hours}, getdate()), getdate()) order by dt, src")

### Преобразование validTill в df

In [169]:
# колонка src - источник
# 1 - данные из orderbook на текущий момент, из памяти
# 2 - данные из исторического orderbook на диске
# 3 - данные из history

df['src'] = df.src.astype('category')
#df['exchange'] = df.src.astype('category')
#df['pair'] = df.src.astype('category')
df['side'] = df.side.astype('category')
df.loc[df.src==1, 'validTill'] = pd.Timestamp.max # преобразуем макс дату из '9999-12-31 23:59:59.000' в Timestamp.max
df['validTill'] = pd.to_datetime(df.validTill)
#df.set_index('dt', inplace=True)
#df['dt'] = pd.to_datetime(df['dt'])
#df.set_index(['dt', 'src'], inplace=True)
#df.index = pd.to_datetime(df.index)

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734625 entries, 0 to 734624
Data columns (total 8 columns):
dt            734625 non-null datetime64[ns]
id_ex_pair    734625 non-null int64
price         734625 non-null float64
amount        734625 non-null float64
side          734625 non-null category
src           734625 non-null category
validFrom     734625 non-null datetime64[ns]
validTill     734625 non-null datetime64[ns]
dtypes: category(2), datetime64[ns](3), float64(2), int64(1)
memory usage: 35.0 MB


In [171]:
# очень медленное преобразование
#df['validTill'] = df['validTill'].apply(lambda x: pd.Timestamp.max if x=='9999-12-31 23:59:59.000' else pd.to_datetime(x))

In [172]:
df.tail(10)

,dt,id_ex_pair,price,amount,side,src,validFrom,validTill
734615,2018-10-12 13:11:40,19,197.720001,4.000000,ask,1,2018-10-12 13:11:41,2262-04-11 23:47:16.854775807
734616,2018-10-12 13:11:40,19,197.429993,6.654700,ask,1,2018-10-12 13:11:41,2262-04-11 23:47:16.854775807
734617,2018-10-12 13:11:40,19,201.970001,0.594150,ask,2,2018-10-12 13:11:41,2018-10-12 13:11:44.000000000
734618,2018-10-12 13:11:40,19,197.220001,0.912690,bid,2,2018-10-12 13:11:41,2018-10-12 13:11:44.000000000
734619,2018-10-12 13:11:43,19,197.119995,1.000000,bid,1,2018-10-12 13:11:44,2262-04-11 23:47:16.854775807
734620,2018-10-12 13:11:43,19,198.509995,25.900000,ask,1,2018-10-12 13:11:44,2262-04-11 23:47:16.854775807
734621,2018-10-12 13:11:43,19,199.270004,21.283310,ask,1,2018-10-12 13:11:44,2262-04-11 23:47:16.854775807
734622,2018-10-12 13:11:46,19,197.740005,0.642820,ask,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807
734623,2018-10-12 13:11:46,19,197.289993,0.053220,bid,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807
734624,2018-10-12 13:11:46,19,197.050003,25.702999,bid,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807


In [173]:
df.describe()

,id_ex_pair,price,amount
count,734625.0,734625.000000,734625.000000
mean,19.0,204.693221,13.130386
std,0.0,10.536954,58.402658
min,19.0,186.610001,0.000010
25%,19.0,198.389999,0.406680
50%,19.0,201.250000,2.000000
75%,19.0,206.259995,9.366860
max,19.0,229.679993,3714.522217


## Поворот Order Book через unstack()

In [174]:
df_pivot=df[['dt','side','price','amount']].groupby(['dt', 'side']).agg({'price': 'last', 'amount': 'sum'}).unstack()

In [175]:
df_pivot.tail(10)

price                                 amount  \
side                        ask         bid         buy sell       ask   
dt                                                                       
2018-10-12 13:11:22  197.339996  195.460007         NaN  NaN  16.21905   
2018-10-12 13:11:24  197.320007  194.339996         NaN  NaN   1.39613   
2018-10-12 13:11:27  197.720001  197.160004         NaN  NaN   4.98382   
2018-10-12 13:11:29         NaN         NaN  197.210007  NaN       NaN   
2018-10-12 13:11:31  197.300003  197.190002         NaN  NaN  19.38056   
2018-10-12 13:11:34  197.350006  197.199997         NaN  NaN   1.00000   
2018-10-12 13:11:37  197.300003  194.470001  197.300003  NaN   6.96129   
2018-10-12 13:11:40  201.970001  197.220001         NaN  NaN  18.25145   
2018-10-12 13:11:43  199.270004  197.119995         NaN  NaN  47.18331   
2018-10-12 13:11:46  197.740005  197.050003         NaN  NaN   0.64282   

                                               
side                        bid      buy sell  
dt                                             
2018-10-12 13:11:22   76.094539      NaN  NaN  
2018-10-12 13:11:24  116.674700      NaN  NaN  
2018-10-12 13:11:27   16.187270      NaN  NaN  
2018-10-12 13:11:29         NaN  0.19545  NaN  
2018-10-12 13:11:31   87.645100      NaN  NaN  
2018-10-12 13:11:34  192.176626      NaN  NaN  
2018-10-12 13:11:37   30.841299  0.12967  NaN  
2018-10-12 13:11:40    5.074980      NaN  NaN  
2018-10-12 13:11:43    1.000000      NaN  NaN  
2018-10-12 13:11:46   25.756219      NaN  NaN

## Группировка OrderBook

In [176]:
df_orderbook = df[df.src.isin([1,2])] #[['dt','side','price','amount']].groupby(['dt', 'side']).agg({'price': 'last', 'amount': 'sum'}).unstack()
df_orderbook.tail()

,dt,id_ex_pair,price,amount,side,src,validFrom,validTill
734620,2018-10-12 13:11:43,19,198.509995,25.900000,ask,1,2018-10-12 13:11:44,2262-04-11 23:47:16.854775807
734621,2018-10-12 13:11:43,19,199.270004,21.283310,ask,1,2018-10-12 13:11:44,2262-04-11 23:47:16.854775807
734622,2018-10-12 13:11:46,19,197.740005,0.642820,ask,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807
734623,2018-10-12 13:11:46,19,197.289993,0.053220,bid,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807
734624,2018-10-12 13:11:46,19,197.050003,25.702999,bid,1,2018-10-12 13:11:47,2262-04-11 23:47:16.854775807


## Подключение OrderBook к History

In [177]:
df_history = df[df.src==3][['dt','side','price','amount']].groupby(['dt', 'side']).agg(
    {'price': 'last', 'amount': 'sum'}).unstack()

In [178]:
df_history.tail(10)

price               amount         
side                        buy        sell      buy     sell
dt                                                           
2018-10-12 13:10:52         NaN  197.300003      NaN  2.97000
2018-10-12 13:10:57         NaN  197.300003      NaN  2.00000
2018-10-12 13:10:58         NaN  197.300003      NaN  0.51000
2018-10-12 13:11:00         NaN  197.289993      NaN  1.97070
2018-10-12 13:11:01         NaN  197.289993      NaN  0.99733
2018-10-12 13:11:04  197.199997         NaN  0.06035      NaN
2018-10-12 13:11:06  197.199997         NaN  0.16400      NaN
2018-10-12 13:11:17  197.199997         NaN  1.59160      NaN
2018-10-12 13:11:29  197.210007         NaN  0.19545      NaN
2018-10-12 13:11:37  197.300003         NaN  0.12967      NaN

In [ ]:
df_history.columns = df_history.columns.droplevel(0)
df_history.columns = ['buy price', 'sell price', 'buy amount', 'sell amount']

### Order book на заданную дату

In [179]:
dt = pd.to_datetime('2018-10-11 11:07:03')

In [180]:
df_orderbook[(dt>=df_orderbook.validFrom) & (dt<df_orderbook.validTill) & (df_orderbook.id_ex_pair==19)].count()

dt            200
id_ex_pair    200
price         200
amount        200
side          200
src           200
validFrom     200
validTill     200
dtype: int64

In [181]:
def get_ob_at_date(df, id_ex_pair, date):
    return df[(date>=df.validFrom) & (date<df.validTill) & (df.id_ex_pair==id_ex_pair)]

In [182]:
# получаем orderbook по ETH/USDT на Binance (19) на заданную дату
df_ob = get_ob_at_date(df_orderbook, 19, dt).sort_values('price')

In [183]:
df_ob.head()

,dt,id_ex_pair,price,amount,side,src,validFrom,validTill
308013,2018-10-11 11:06:59,19,198.639999,1.01188,bid,2,2018-10-11 11:07:01,2018-10-11 11:07:04
308021,2018-10-11 11:06:59,19,198.649994,0.11347,bid,2,2018-10-11 11:07:01,2018-10-11 11:07:10
308016,2018-10-11 11:06:59,19,198.750000,3.77245,bid,2,2018-10-11 11:07:01,2018-10-11 11:12:25
306472,2018-10-11 11:03:17,19,198.800003,1.33276,bid,2,2018-10-11 11:03:18,2018-10-11 11:13:19
296186,2018-10-11 10:33:45,19,198.839996,3.47909,bid,2,2018-10-11 10:33:46,2018-10-11 11:13:22


In [346]:
df_ob.describe()

,id_ex_pair,price,amount
count,200.0,200.000000,200.000000
mean,19.0,202.031250,15.062258
std,0.0,2.061227,44.163498
min,19.0,198.639999,0.000020
25%,19.0,200.240002,0.180930
50%,19.0,201.805000,1.002005
75%,19.0,203.825001,7.265513
max,19.0,205.800003,286.809265


In [185]:
# разделяем биды и аски по 100 штук в каждом
bids = df_ob[df_ob.side=='bid']['price'].values[-100:]
asks = df_ob[df_ob.side=='ask']['price'].values[:100]
bid_amounts = df_ob[df_ob.side=='bid']['amount'].values[-100:]
ask_amounts = df_ob[df_ob.side=='ask']['amount'].values[:100]

In [186]:
# показываем 10 ближних бидов
bids[-10:]

array([201.58000183, 201.58999634, 201.6000061 , 201.66000366,
       201.66999817, 201.69999695, 201.72000122, 201.72999573,
       201.74000549, 201.75999451])

In [187]:
# цену
df_history.loc[dt].price.dropna()[0]

201.85000610351562

In [188]:
# и 10 ближних асков
asks[:10]

array([201.8500061 , 201.86000061, 201.86999512, 201.88000488,
       201.88999939, 201.92999268, 201.94000244, 201.96000671,
       201.97000122, 202.        ])

In [398]:
def get_bids(date):
    id_ex_pair = 19
    df = df_orderbook
    #df_ob = df[(date>=df.validFrom) & (date<df.validTill) & (df.id_ex_pair==id_ex_pair)].sort_values('price')
    df_ob = df.iloc[np.where((dt>=df.validFrom) & (dt<df.validTill) & (df.id_ex_pair==19))[0],:].sort_values('price')
    
    # разделяем биды и аски по 100 штук в каждом
    bids = df_ob[df_ob.side=='bid']['price'].values[-100:]
    #asks = df_ob[df_ob.side=='ask']['price'].values[:100]
    #bid_amounts = df_ob[df_ob.side=='bid']['amount'].values[-100:]
    #ask_amounts = df_ob[df_ob.side=='ask']['amount'].values[:100]
    return bids #, asks #, bid_amounts, ask_amounts

In [399]:
get_bids(date=dt)[:10] # id_ex_pair=19,

array([198.63999939, 198.6499939 , 198.75      , 198.80000305,
       198.83999634, 198.86000061, 198.91000366, 198.99000549,
       199.        , 199.00999451])

In [400]:
df_orderbook.head()

,dt,id_ex_pair,price,amount,side,src,validFrom,validTill
0,2018-10-10 13:11:48,19,226.330002,1.81063,bid,2,2018-10-10 13:11:49,2018-10-10 13:12:10
1,2018-10-10 13:11:48,19,224.039993,0.68876,bid,2,2018-10-10 13:11:49,2018-10-10 13:11:52
2,2018-10-10 13:11:48,19,225.979996,1.05000,bid,2,2018-10-10 13:11:49,2018-10-10 13:11:52
3,2018-10-10 13:11:48,19,226.479996,10.16653,ask,2,2018-10-10 13:11:49,2018-10-10 13:12:05
4,2018-10-10 13:11:48,19,226.429993,1.81047,ask,2,2018-10-10 13:11:49,2018-10-10 13:11:55


In [318]:
#dfv = np.vectorize(get_bids)(df_orderbook, id_ex_pair=19, date=df_history.index)
df_history.head()

,buy price,sell price,buy amount,sell amount
dt,,,,
2018-10-10 13:11:57,226.440002,NaN,18.754519,NaN
2018-10-10 13:11:59,NaN,226.350006,NaN,0.74759
2018-10-10 13:12:04,226.419998,NaN,0.100000,NaN
2018-10-10 13:12:07,226.440002,NaN,21.072360,NaN
2018-10-10 13:12:10,NaN,226.369995,NaN,1.35080


In [401]:
pd.to_datetime(df_history.iloc[:10].index.values)

DatetimeIndex(['2018-10-10 13:11:57', '2018-10-10 13:11:59',
               '2018-10-10 13:12:04', '2018-10-10 13:12:07',
               '2018-10-10 13:12:10', '2018-10-10 13:12:11',
               '2018-10-10 13:12:13', '2018-10-10 13:12:15',
               '2018-10-10 13:12:17', '2018-10-10 13:12:27'],
              dtype='datetime64[ns]', freq=None)

In [397]:
# df_history.head(10).apply(lambda x: get_bids(19, pd.to_datetime(x.index)), axis=1)
# np.vectorize(get_bids)(19, pd.to_datetime(df_history.index.values).astype(str))
# df_history.reset_index().head().applymap(lambda x: get_bids(19, '2018-10-10 13:12:11'))
# df_history.head().applymap(lambda x: get_bids(19, pd.to_datetime(x.index)))
df_history.reset_index().head()['dt'].apply(get_bids)

0    [225.97999572753906, 225.99000549316406, 226.3...
1    [225.97999572753906, 225.99000549316406, 226.3...
2    [224.05999755859375, 224.0800018310547, 224.13...
3    [224.0500030517578, 224.05999755859375, 224.08...
4    [224.05999755859375, 224.0800018310547, 224.13...
Name: dt, dtype: object

In [336]:
df_history.reset_index().head()

,dt,buy price,sell price,buy amount,sell amount
0,2018-10-10 13:11:57,226.440002,NaN,18.754519,NaN
1,2018-10-10 13:11:59,NaN,226.350006,NaN,0.74759
2,2018-10-10 13:12:04,226.419998,NaN,0.100000,NaN
3,2018-10-10 13:12:07,226.440002,NaN,21.072360,NaN
4,2018-10-10 13:12:10,NaN,226.369995,NaN,1.35080


In [402]:
id_ex_pair = 19
dt = pd.to_datetime('2018-10-11 11:07:03')
df_orderbook.iloc[np.where((dt>=df_orderbook.validFrom) & (dt<df_orderbook.validTill) & 
                           (df_orderbook.id_ex_pair==19))[0],:].head()

,dt,id_ex_pair,price,amount,side,src,validFrom,validTill
294896,2018-10-11 10:30:39,19,203.279999,0.10000,ask,2,2018-10-11 10:30:40,2018-10-11 14:58:22
294907,2018-10-11 10:30:39,19,203.210007,0.09843,ask,2,2018-10-11 10:30:40,2018-10-11 11:07:07
295434,2018-10-11 10:31:57,19,203.289993,0.26364,ask,2,2018-10-11 10:31:58,2018-10-11 11:15:58
295500,2018-10-11 10:32:03,19,203.550003,5.28630,ask,2,2018-10-11 10:32:04,2018-10-11 11:23:43
295503,2018-10-11 10:32:03,19,203.440002,16.72789,ask,2,2018-10-11 10:32:04,2018-10-11 11:13:28


In [413]:
df_history.reset_index(inplace=True)

In [418]:
df_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75580 entries, 0 to 75579
Data columns (total 6 columns):
dt             75580 non-null datetime64[ns]
buy price      50210 non-null float64
sell price     45530 non-null float64
buy amount     50210 non-null float64
sell amount    45530 non-null float64
bids           1000 non-null object
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 3.5+ MB


In [419]:
%timeit -n 1 -r 1 df_history['bids'] = df_history.head(1000)['dt'].map(get_bids) #---SLOW!

5.71 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [354]:
get_bids_vectorized = np.vectorize(get_bids, otypes=[np.datetime64])

In [356]:
#df_history['bids'] = get_bids_vectorized(df_history.reset_index()['dt'])

In [377]:
df_history.head()

,buy price,sell price,buy amount,sell amount,bids
dt,,,,,
2018-10-10 13:11:57,226.440002,NaN,18.754519,NaN,NaN
2018-10-10 13:11:59,NaN,226.350006,NaN,0.74759,NaN
2018-10-10 13:12:04,226.419998,NaN,0.100000,NaN,NaN
2018-10-10 13:12:07,226.440002,NaN,21.072360,NaN,NaN
2018-10-10 13:12:10,NaN,226.369995,NaN,1.35080,NaN


In [344]:
df = df_orderbook.copy()
id_ex_pair = 19
dt = pd.to_datetime('2018-10-11 11:07:03')

np.where(
    df[(dt>=df.validFrom) & (dt<df.validTill) & (df.id_ex_pair==id_ex_pair) & 
       (df.side=='bid')] #.sort_values('price')['price'].values[-100:]
)

AttributeError: 'DataFrame' object has no attribute 'is_bid'